## Step 4 Create Final Network Graph and Assign Link Costs
---
In this notebook, the reconciled network is finalized and link costs are assigned based on link attributes.

## Import modules

In [1]:
from pathlib import Path
import geopandas as gpd
from prepare_network import *
from network_filter import *

## Prepare network by removing isolates and creating a directed network graph

### NCST Transit Portion

In [2]:
export_fp = Path.home() / 'Documents/TransitSimData/networks'

#bring in network
links = gpd.read_file(export_fp / 'reconciled_network.gpkg',layer='links')
nodes = gpd.read_file(export_fp / 'reconciled_network.gpkg',layer='nodes')

#prepare network
links, nodes = prepare_network(links,nodes,spd_mph=8)

#if just using time impedance, export as is
nodes.to_file(export_fp/'final_network.gpkg',layer='nodes',driver='GPKG')
links.to_file(export_fp/'final_network.gpkg',layer='links',driver='GPKG')

#repeat for low stress network
links = gpd.read_file(export_fp / 'lowstress_network.gpkg',layer='links')
nodes = gpd.read_file(export_fp / 'lowstress_network.gpkg',layer='nodes')

#prepare network
links, nodes = prepare_network(links,nodes,spd_mph=8)

#if just using time impedance, export as is
nodes.to_file(export_fp/'final_network.gpkg',layer='lowstress_nodes',driver='GPKG')
links.to_file(export_fp/'final_network.gpkg',layer='lowstress_links',driver='GPKG')

### NCST Bike Portion (continues on to applying link costs)

In [3]:
export_fp = Path.home() / Path(f'Documents/NewBikewaySimData/bikewaysim')

#bring in network
links = gpd.read_file(export_fp / 'reconciled_network.gpkg',layer='links')
nodes = gpd.read_file(export_fp / 'reconciled_network.gpkg',layer='nodes')

#prepare network
links, nodes = prepare_network(links,nodes,spd_mph=8)

NameError: name 'studyarea_name' is not defined

## Applying Link Costs
---
Dict keys must correspond to column names in links GeoDataFrame. Multiple dicts can be passed to test the impacts of changing impedances. The links cost function is of this format:
$$ C_e = \frac{l_e*60^2}{s*5280} * (1-\sum \beta_i x_{i,e}) $$

where:
- $e$ is an edge/link in network graph $G$ with V vertices/nodes and E edges/links
- $l_e$ is the length of the link in feet
- $\beta$ is the impedance coefficient for attribute $i$
- $X_{i,e}$ is the value of attribute $i$ for link $e$
- $s$ is the assumed average speed of the cyclist in mph

Notes:
- Negative attributes **decrease** impedance  
- Positive attributes **increase** impedance
- **Negative link costs are not allowed**
- Time to traverse a link has already been calculated in the prepare_network function

In [ ]:
costs0 = {
    'bl':-0.05,
    'pbl':-0.95,
    'mu':-0.95,
    '<25mph':0,
    '25-30mph':0.30,
    '>30mph':1,
    '1lpd':0,
    '2-3lpd':0.50,
    '>4lpd':1     
    }

In [ ]:
costs1 = {
    'bl':-0.25,
    'pbl':-0.60,
    'mu':-0.60,
    '<25mph':0,
    '25-30mph':0.50,
    '>30mph':1.5,
    '1lpd':0,
    '2-3lpd':0.50,
    '>4lpd':1.5     
    }

In [ ]:
costs2 = {
    'bl':-0.05,
    'pbl':-0.5,
    'mu':-0.5,
    '<25mph':0,
    '25-30mph':0.5,
    '>30mph':4,
    '1lpd':0,
    '2-3lpd':1,
    '>4lpd':4     
    }

In [ ]:
cost_dicts = {
    'costs0':costs0,
    'costs1':costs1,
    'costs2':costs2
}

### Calculate link costs and export results (i.e. where did link impedances increase/decrease/both/stay the same) for visualization

In [ ]:
for key in cost_dicts.keys():

    #other costs to test
    links = link_costs(links, cost_dicts[key], key)

    #make fp organizing
    if not (export_fp / Path(key)).exists():
        (export_fp / Path(key)).mkdir()

    positive_cols = [x for x in cost_dicts[key].keys() if cost_dicts[key][x] < 0]
    nochange_cols = [x for x in cost_dicts[key].keys() if cost_dicts[key][x] == 0]
    negative_cols = [x for x in cost_dicts[key].keys() if cost_dicts[key][x] > 0]

    positive = (links[positive_cols] == 1).any(axis=1)
    negative = (links[negative_cols] == 1).any(axis=1)
    no_change = (links[nochange_cols] == 1).any(axis=1)

    only_positive = positive & -negative
    only_negative = -positive & negative
    mixed = positive & negative #& -no_change
    only_no_change = no_change & -positive & -negative

    only_positive = links[only_positive]
    only_negative = links[only_negative]
    mixed = links[mixed]
    no_change = links[only_no_change]

    only_positive.to_file(export_fp / key / 'link_cost_changes.gpkg',layer='Only Decreased')
    only_negative.to_file(export_fp / key / 'link_cost_changes.gpkg',layer='Only Increased')
    mixed.to_file(export_fp / key / 'link_cost_changes.gpkg',layer='Mixed')
    no_change.to_file(export_fp / key / 'link_cost_changes.gpkg',layer='No Change')

#export network with costs to file
nodes.to_file(export_fp/'final_network.gpkg',layer='nodes',driver='GPKG')
links.to_file(export_fp/'final_network.gpkg',layer='links',driver='GPKG')

## Network Improvements
---
Use this section to add new links or improve existing ones. New links should be drawn in a GIS program. Functions for adding reference columns and splitting existing links will be added in the future, but need to be manually added for now. If wanting to see the impacts of only one improvement/new link, be sure to export that as a separate network.

### Improving an existing link with a protected bike lane

In [ ]:
#extend protected bike lane on 10th st
improvements = links.copy()
changes = ['10th Street Northwest','10th Street Northeast']
improvements.loc[improvements['name'].isin(changes),'pbl'] = 1

#change a specific link back
improvements.loc[improvements['A_B']=='9543446970_69614574','pbl'] = 0
improvements.loc[improvements['A_B']=='69614574_9543446970','pbl'] = 0

#export the new improvement for mapping/visualizations
tenth_st  = improvements[improvements['pbl'] != links['pbl']].copy()
tenth_st.geometry = tenth_st.buffer(300)
tenth_st = tenth_st.dissolve()
tenth_st.to_file(export_fp/'network_improvements.gpkg',layer='New Protected Bike Lane')

#create a copy so the improvement from the protected bike lane can be tested separately
just_pbl = improvements.copy()

### Bring in feature(s) drawn in GIS and add to network

In [ ]:
#bring in new feature and add to old links
new = gpd.read_file(export_fp/'network_improvements.gpkg',layer='morningside_path')
morningside_path = new.copy()
morningside_path.geometry = morningside_path.buffer(300)
morningside_path = morningside_path.dissolve()
morningside_path.to_file(export_fp / "network_improvements.gpkg", layer = 'New Multi-Use Path')

#add ref ids ONLY if no new nodes are added
new = add_ref_ids(new,nodes)

#make reverse links if they are two way features
new = create_reverse_links(new)
new['dist'] = new.length
new['mins'] = new['dist'] / 5280 / 8 * 60

#add to network with 10st pbl
improvements = pd.concat([improvements,new],ignore_index=True)

#just add path
just_path = links.copy()
just_path = pd.concat([just_path,new],ignore_index=True)

c:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Reference IDs successfully added to links.


### Apply new costs after improvements and export

In [ ]:
#redo costs
for key in cost_dicts.keys():
    #other costs to test
    just_pbl = link_costs(just_pbl, cost_dicts[key], key)
    just_path = link_costs(just_path, cost_dicts[key], key)
    improvements = link_costs(improvements, cost_dicts[key], key)

#export
just_pbl.to_file(export_fp / 'final_network.gpkg',layer='just_pbl_links')
just_path.to_file(export_fp / 'final_network.gpkg',layer='just_path_links')
improvements.to_file(export_fp / 'final_network.gpkg',layer='improved_links')